In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/mwitiderrick/stockprice/master/NSE-TATAGLOBAL.csv")

data.head()

,Date,Open,High,Low,Last,Close,Total Trade Quantity,Turnover (Lacs)
0,2018-09-28,234.05,235.95,230.20,233.50,233.75,3069914,7162.35
1,2018-09-27,234.55,236.80,231.10,233.80,233.25,5082859,11859.95
2,2018-09-26,240.00,240.00,232.50,235.00,234.25,2240909,5248.60
3,2018-09-25,233.30,236.75,232.00,236.25,236.10,2349368,5503.90
4,2018-09-24,233.55,239.20,230.75,234.00,233.30,3423509,7999.55


In [ ]:
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

In [ ]:
data = data.sort_index()

In [ ]:
data.drop(['Total Trade Quantity', 'Turnover (Lacs)'], axis=1, inplace=True)


In [ ]:
train_size = 0.8
train_data = data[ :int(len(data)*0.8)]
test_data = data[int(len(data)*0.8): ]


In [ ]:
scaler = MinMaxScaler()
train_data_scaled = scaler.fit_transform(train_data)
test_data_scaled = scaler.transform(test_data)

In [ ]:
timesteps = 3
n_features = 4  

train_X, train_y = [], []
for i in range(timesteps, len(train_data_scaled)):
    train_X.append(train_data_scaled[i-timesteps:i, :n_features])  
    train_y.append(train_data_scaled[i, -1])
train_X, train_y = np.array(train_X), np.array(train_y)

In [ ]:

test_X, test_y = [], []
for i in range(timesteps, len(test_data_scaled)):
    test_X.append(test_data_scaled[i-timesteps:i, :n_features])
    test_y.append(test_data_scaled[i, -1])
test_X, test_y = np.array(test_X), np.array(test_y)

In [ ]:
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(timesteps, 4)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')

In [ ]:
history = model.fit(train_X, train_y, epochs=50, batch_size=64, validation_data=(test_X, test_y), verbose=1)

Epoch 1/50
26/26 [==============================] - 1s 27ms/step - loss: 7.8772e-04 - val_loss: 0.0172
Epoch 2/50
26/26 [==============================] - 1s 28ms/step - loss: 8.0460e-04 - val_loss: 0.0183
Epoch 3/50
26/26 [==============================] - 1s 19ms/step - loss: 8.0094e-04 - val_loss: 0.0208
Epoch 4/50
26/26 [==============================] - 0s 16ms/step - loss: 7.8478e-04 - val_loss: 0.0160
Epoch 5/50
26/26 [==============================] - 0s 15ms/step - loss: 7.9471e-04 - val_loss: 0.0235
Epoch 6/50
26/26 [==============================] - 1s 20ms/step - loss: 7.7577e-04 - val_loss: 0.0096
Epoch 7/50
26/26 [==============================] - 0s 17ms/step - loss: 7.7862e-04 - val_loss: 0.0118
Epoch 8/50
26/26 [==============================] - 1s 24ms/step - loss: 8.4285e-04 - val_loss: 0.0082
Epoch 9/50
26/26 [==============================] - 1s 21ms/step - loss: 7.8750e-04 - val_loss: 0.0235
Epoch 10/50
26/26 [==============================] - 0s 19ms/step - loss:

In [ ]:
mse = model.evaluate(test_X, test_y)
print('Mean Squared Error:', mse)

13/13 [==============================] - 0s 3ms/step - loss: 0.0130
Mean Squared Error: 0.012959010899066925
